# Eat Safe, Love

## Notebook Set Up

In [112]:
# Dependencies
from pymongo import MongoClient
from bson import ObjectId 
import pandas as pd
from pprint import pprint

In [113]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [114]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [115]:
# review the collections in our database
collections = db.list_collection_names()
collections

['establishments']

In [116]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question:
    * Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [117]:
# review a document in the establishments collection
doc_one = establishments.find_one()
pprint(doc_one)

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('66ac553db2f827193321d168'),
 'geocode': {'latitude': 51.083812, 'longitude': 1.195625},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': None,


In [118]:
# Find the establishments with a hygiene score of 20
query = {
        'scores.Hygiene': 20
}

results = list(establishments.find(query))

# Display the first document in the results using pprint
pprint(results[0])

{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': 0,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('66ac553fb2f827193321ec9a'),
 'geocode': {'latitude': 50.769705, 'longitude': 0.27694},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/110681',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate

In [119]:
# Use count_documents to display the number of documents in the result
cnt_results = establishments.count_documents(query)
cnt_results

41

In [120]:
# Define a flattten function to handle the json file
def flatten_records(record):
    # Create a copy of original data, so any modification wouldn't affect the original data
    flat_records = record.copy()
    
    # Handle the geocode fields
    if 'geocode' in record:
        flat_records['latitude'] = record['geocode'].get('latitude')
        flat_records['longitude'] = record['geocode'].get('longitude')
        del flat_records['geocode']      
        
    # Handle the scores fields
    if 'scores' in record:
        for key, value in flat_records['scores'].items():
            flat_records[f'{key}_scores'] = value
        del flat_records['scores']
    
    # Handle the meta fields
    if 'meta' in record:
        for key, value in flat_records['meta'].items():
            flat_records[f'{key}_meta'] = value
        del flat_records['meta']
    
    # Hanlde the links fields
    # Check the links fields exists and at least contains one element.
    if 'links' in record and isinstance(record['links'], list) and len(record['links']) > 0:
        flat_records['links_ref'] = record['links'][0].get('href')  # Only keep the website links
        del flat_records['links']
    
    return flat_records

# Processing the json file
processed_records = [flatten_records(record) for record in results]

# Convert the result to a Pandas DataFrame
hygiene_df = pd.DataFrame(processed_records)

#review the DataFrame
hygiene_df

,_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,...,ConfidenceInManagement_scores,dataSource_meta,extractDate_meta,itemCount_meta,returncode_meta,totalCount_meta,totalPages_meta,pageSize_meta,pageNumber_meta,links_ref
0,66ac553fb2f827193321ec9a,110681,0,4029,The Chase Rest Home,Caring Premises,5,5-6 Southfields Road,Eastbourne,East Sussex,...,20,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...
1,66ac553fb2f827193321f01b,612039,0,1970/FOOD,Brenalwood,Caring Premises,5,Hall Lane,Walton-on-the-Naze,Essex,...,30,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...
2,66ac553fb2f827193321f325,730933,0,1698/FOOD,Melrose Hotel,Hotel/bed & breakfast/guest house,7842,53 Marine Parade East,Clacton On Sea,Essex,...,20,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...
3,66ac553fb2f827193321f513,172735,0,PI/000023858,Seaford Pizza,Takeaway/sandwich shop,7844,4 High Street,Seaford,East Sussex,...,20,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...
4,66ac553fb2f827193321f522,172953,0,PI/000024532,Golden Palace,Restaurant/Cafe/Canteen,1,5 South Street,Seaford,East Sussex,...,20,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...
5,66ac5540b2f827193321fec1,512854,0,12/00816/BUTH,Ashby's Butchers,Retailers - other,4613,777 Southchurch Road,Southend-On-Sea,Essex,...,20,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...
6,66ac5540b2f82719332200e2,1537089,0,22/00224/RESTUN,South Sea Express Cuisine,Restaurant/Cafe/Canteen,1,33 Alexandra Street,Southend-on-sea,Essex,...,20,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...
7,66ac5541b2f827193322160d,155648,0,EH/00006058,Golden Palace,Takeaway/sandwich shop,7844,7 London Road,Rayleigh,Essex,...,30,None,0001-01-01T00:00:00,0,None,0,0,0,0,http://api.ratings.food.gov.uk/establishments/...
8,66ac5542b2f8271933221a51,1012883,0,17/00110/MIXED/S,The Tulip Tree,Restaurant/Cafe/Canteen,1,3 The Village,Chiddingstone,KENT,...,20,None,0001-01-01T00:00:00,0,None,0,0,0,0,http://api.ratings.food.gov.uk/establishments/...
9,66ac5542b2f8271933222263,644109,0,41811,F & S,Retailers - other,4613,,81 Southernhay,Basildon,...,20,None,0001-01-01T00:00:00,0,None,0,0,0,0,http://api.ratings.food.gov.uk/establishments/...


In [121]:
# Display the number of rows in the DataFrame
num_hygiene_df = hygiene_df.shape[0]
print(f'Hygiene records has {num_hygiene_df} rows.')

Hygiene records has 41 rows.


In [122]:
# Display the first 10 rows of the DataFrame
hygiene_df.head(10)

,_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,...,ConfidenceInManagement_scores,dataSource_meta,extractDate_meta,itemCount_meta,returncode_meta,totalCount_meta,totalPages_meta,pageSize_meta,pageNumber_meta,links_ref
0,66ac553fb2f827193321ec9a,110681,0,4029,The Chase Rest Home,Caring Premises,5,5-6 Southfields Road,Eastbourne,East Sussex,...,20,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...
1,66ac553fb2f827193321f01b,612039,0,1970/FOOD,Brenalwood,Caring Premises,5,Hall Lane,Walton-on-the-Naze,Essex,...,30,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...
2,66ac553fb2f827193321f325,730933,0,1698/FOOD,Melrose Hotel,Hotel/bed & breakfast/guest house,7842,53 Marine Parade East,Clacton On Sea,Essex,...,20,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...
3,66ac553fb2f827193321f513,172735,0,PI/000023858,Seaford Pizza,Takeaway/sandwich shop,7844,4 High Street,Seaford,East Sussex,...,20,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...
4,66ac553fb2f827193321f522,172953,0,PI/000024532,Golden Palace,Restaurant/Cafe/Canteen,1,5 South Street,Seaford,East Sussex,...,20,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...
5,66ac5540b2f827193321fec1,512854,0,12/00816/BUTH,Ashby's Butchers,Retailers - other,4613,777 Southchurch Road,Southend-On-Sea,Essex,...,20,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...
6,66ac5540b2f82719332200e2,1537089,0,22/00224/RESTUN,South Sea Express Cuisine,Restaurant/Cafe/Canteen,1,33 Alexandra Street,Southend-on-sea,Essex,...,20,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...
7,66ac5541b2f827193322160d,155648,0,EH/00006058,Golden Palace,Takeaway/sandwich shop,7844,7 London Road,Rayleigh,Essex,...,30,None,0001-01-01T00:00:00,0,None,0,0,0,0,http://api.ratings.food.gov.uk/establishments/...
8,66ac5542b2f8271933221a51,1012883,0,17/00110/MIXED/S,The Tulip Tree,Restaurant/Cafe/Canteen,1,3 The Village,Chiddingstone,KENT,...,20,None,0001-01-01T00:00:00,0,None,0,0,0,0,http://api.ratings.food.gov.uk/establishments/...
9,66ac5542b2f8271933222263,644109,0,41811,F & S,Retailers - other,4613,,81 Southernhay,Basildon,...,20,None,0001-01-01T00:00:00,0,None,0,0,0,0,http://api.ratings.food.gov.uk/establishments/...


In [123]:
# Save the data into a csv file
output_file_path = './Outputs_Data/hygiene_records.csv'
hygiene_df.to_csv(output_file_path)

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [124]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.

query = {
    'RatingValue' : {'$gte': 4}
}

rating_value_results = list(establishments.find(query))

# Display the first document in the results using pprint
pprint(rating_value_results[0])

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('66ac553db2f827193321d168'),
 'geocode': {'latitude': 51.083812, 'longitude': 1.195625},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': None,


In [106]:
# Use count_documents to display the number of documents in the result
cnt_rating_results = establishments.count_documents(query)
print(f'There are {cnt_rating_results} records have ratingvalue greater than or equal 4.')

There are 31257 records have ratingvalue greater than or equal 4.


In [125]:
# Using the same method to processing the json file
rating_processing_results = [flatten_records(record) for record in rating_value_results]

# Convert the result to a Pandas DataFrame
rating_df = pd.DataFrame(rating_processing_results)
rating_df

,_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,...,ConfidenceInManagement_scores,dataSource_meta,extractDate_meta,itemCount_meta,returncode_meta,totalCount_meta,totalPages_meta,pageSize_meta,pageNumber_meta,links_ref
0,66ac553db2f827193321d168,1043695,0,PI/000073616,The Pavilion,Restaurant/Cafe/Canteen,1,East Cliff Pavilion,Wear Bay Road,Folkestone,...,5.0,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...
1,66ac553db2f827193321d169,647177,0,PI/000041489,Wear Bay Bowls Club,Pub/bar/nightclub,7843,Wear Bay Road,Folkestone,Kent,...,10.0,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...
2,66ac553db2f827193321d16a,289353,0,PI/000002468,St Marys COE (aided) Primary School,School/college/university,7845,St Marys Church Of England Primary School,Warren Road,Folkestone,...,0.0,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...
3,66ac553db2f827193321d16d,289352,0,PI/000002460,The Ship,Pub/bar/nightclub,7843,23 The Stade,Folkestone,Kent,...,5.0,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...
4,66ac553db2f827193321d16e,289560,0,PI/000039927,Mariner,Pub/bar/nightclub,7843,16 The Stade,Folkestone,Kent,...,0.0,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31252,66ac5547b2f82719332269de,961078,0,708124,Barley C of E School,School/college/university,7845,Barley Church Of England Vc First School Churc...,,Barley,...,0.0,None,0001-01-01T00:00:00,0,None,0,0,0,0,http://api.ratings.food.gov.uk/establishments/...
31253,66ac5547b2f82719332269e0,961141,0,708525,Barkway First School,School/college/university,7845,Barkway First School 84 High Street,,Barkway,...,0.0,None,0001-01-01T00:00:00,0,None,0,0,0,0,http://api.ratings.food.gov.uk/establishments/...
31254,66ac5547b2f82719332269e1,1109561,0,715931,The Fox & Hounds,Pub/bar/nightclub,7843,Fox And Hounds High Street,,Barley,...,5.0,None,0001-01-01T00:00:00,0,None,0,0,0,0,http://api.ratings.food.gov.uk/establishments/...
31255,66ac5547b2f82719332269e2,1424837,0,718267,La Vita,Restaurant/Cafe/Canteen,1,4 High Street,,Barkway,...,0.0,None,0001-01-01T00:00:00,0,None,0,0,0,0,http://api.ratings.food.gov.uk/establishments/...


In [126]:
# Using shape to calculate the number of records
num_rating_df = rating_df.shape[0]

# Display the number of rows in the DataFrame
print(f'There are {num_rating_df} records have rating value greater than or equal 4.')

There are 31257 records have rating value greater than or equal 4.


In [127]:
# Display the first 10 rows of the DataFrame
rating_df.head(10)

,_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,...,ConfidenceInManagement_scores,dataSource_meta,extractDate_meta,itemCount_meta,returncode_meta,totalCount_meta,totalPages_meta,pageSize_meta,pageNumber_meta,links_ref
0,66ac553db2f827193321d168,1043695,0,PI/000073616,The Pavilion,Restaurant/Cafe/Canteen,1,East Cliff Pavilion,Wear Bay Road,Folkestone,...,5.0,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...
1,66ac553db2f827193321d169,647177,0,PI/000041489,Wear Bay Bowls Club,Pub/bar/nightclub,7843,Wear Bay Road,Folkestone,Kent,...,10.0,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...
2,66ac553db2f827193321d16a,289353,0,PI/000002468,St Marys COE (aided) Primary School,School/college/university,7845,St Marys Church Of England Primary School,Warren Road,Folkestone,...,0.0,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...
3,66ac553db2f827193321d16d,289352,0,PI/000002460,The Ship,Pub/bar/nightclub,7843,23 The Stade,Folkestone,Kent,...,5.0,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...
4,66ac553db2f827193321d16e,289560,0,PI/000039927,Mariner,Pub/bar/nightclub,7843,16 The Stade,Folkestone,Kent,...,0.0,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...
5,66ac553db2f827193321d16f,987206,0,PI/000075655,The Ships Galley,Pub/bar/nightclub,7843,The Ship,23 The Stade,Folkestone,...,10.0,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...
6,66ac553db2f827193321d170,344689,0,PI/000002095,Folkestone Trawlers Shop,Retailers - other,4613,1 The Stade,Folkestone,Kent,...,5.0,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...
7,66ac553db2f827193321d171,894592,0,PI/000074396,Docker,Retailers - other,4613,Folkestone Harbour,Harbour Approach Road,Folkestone,...,5.0,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...
8,66ac553db2f827193321d172,1043701,0,PI/000076360,Pick Up Pintxos,Takeaway/sandwich shop,7844,Folkestone Harbour,Harbour Approach Road,Folkestone,...,5.0,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...
9,66ac553db2f827193321d173,805702,0,PI/000073219,Dr Legumes - Harbour Arm,Restaurant/Cafe/Canteen,1,Folkestone Harbour,Harbour Approach Road,Folkestone,...,10.0,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...


In [128]:
# Save the DataFrame into a csv file.
output_file_path = './Outputs_Data/rating_records.csv'
rating_df.to_csv(output_file_path)

3. What are the top 5 establishments with a RatingValue rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [129]:
# Query the Penang Flavours's data, only return restaurant name, latitude, longtitude.
query = {
    'BusinessName': 'Penang Flavours'
}

fileds = {
    '_id':1,
    'BusinessName':1,
    'geocode.latitude':1,
    'geocode.longitude':1
}

penang_data = list(establishments.find(query, fileds))
pprint(penang_data)

[{'BusinessName': 'Penang Flavours',
  '_id': ObjectId('66ac555958a546f85caa0004'),
  'geocode': {'latitude': 51.490142, 'longitude': 0.08384}}]


In [132]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score 

degree_search = 0.01
latitude = float(penang_data[0]['geocode']['latitude'])
longitude = float(penang_data[0]['geocode']['longitude']) 
# print(latitude, longitude) 

# Search criteria: Latitude and longitude within a range of 0.01 degrees.
lat_range  = [latitude - degree_search, latitude + degree_search]
lon_range = [longitude - degree_search, longitude + degree_search]
# print(lat_range)
# print(lon_range)
match_query = {
    '$match': {
        'geocode.latitude': {'$gte': lat_range[0], '$lte': lat_range[1]},
        'geocode.longitude': {'$gte': lon_range[0], '$lte': lon_range[1]},
        'RatingValue': 5
    }
}



sort_values = {
    '$sort': {
        'scores.Hygiene': 1
    }
}

limit_values = {
    '$limit': 5
}

# Put the pipeline together
pipeline = [match_query, sort_values, limit_values]

# Run the pipeline, cast the results as a list
top_restaurants_results = list(establishments.aggregate(pipeline))

# Print the results
top_restaurants_results

[{'_id': ObjectId('66ac5544b2f82719332244dc'),
  'FHRSID': 1380578,
  'ChangesByServerID': 0,
  'LocalAuthorityBusinessID': '14425',
  'BusinessName': 'Howe and Co Fish and Chips - Van 17',
  'BusinessType': 'Mobile caterer',
  'BusinessTypeID': 7846,
  'AddressLine1': 'Restaurant And Premises 107A Plumstead High Street',
  'AddressLine2': '',
  'AddressLine3': 'Plumstead',
  'AddressLine4': 'Greenwich',
  'PostCode': 'SE18 1SE',
  'Phone': '',
  'RatingValue': 5,
  'RatingKey': 'fhrs_5_en-gb',
  'RatingDate': '2021-11-11T00:00:00',
  'LocalAuthorityCode': '511',
  'LocalAuthorityName': 'Greenwich',
  'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
  'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
  'scores': {'Hygiene': 0, 'Structural': 0, 'ConfidenceInManagement': 0},
  'SchemeType': 'FHRS',
  'geocode': {'longitude': 0.0925370007753372, 'latitude': 51.4875335693359},
  'RightToReply': '',
  'Distance': 4646.95593107927,
  'NewRatingPending': False,
  'meta

In [133]:
# Using the same method to processing the json file
top5_processing_results = [flatten_records(record) for record in top_restaurants_results]

# Convert result to Pandas DataFrame
top5_restaurants_df = pd.DataFrame(top5_processing_results)
top5_restaurants_df

,_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,...,ConfidenceInManagement_scores,dataSource_meta,extractDate_meta,itemCount_meta,returncode_meta,totalCount_meta,totalPages_meta,pageSize_meta,pageNumber_meta,links_ref
0,66ac5544b2f82719332244dc,1380578,0,14425,Howe and Co Fish and Chips - Van 17,Mobile caterer,7846,Restaurant And Premises 107A Plumstead High St...,,Plumstead,...,0,None,0001-01-01T00:00:00,0,None,0,0,0,0,http://api.ratings.food.gov.uk/establishments/...
1,66ac5544b2f8271933224514,694478,0,PI/000086506,Atlantic Fish Bar,Takeaway/sandwich shop,7844,35 Lakedale Road,,Plumstead,...,5,None,0001-01-01T00:00:00,0,None,0,0,0,0,http://api.ratings.food.gov.uk/establishments/...
2,66ac5544b2f8271933224511,695241,0,PI/000179088,Plumstead Manor Nursery,Caring Premises,5,Plumstead Manor School Old Mill Road,,Plumstead,...,5,None,0001-01-01T00:00:00,0,None,0,0,0,0,http://api.ratings.food.gov.uk/establishments/...
3,66ac5544b2f82719332244cc,695223,0,PI/000178842,Iceland,Retailers - supermarkets/hypermarkets,7840,144 - 146 Plumstead High Street,,Plumstead,...,5,None,0001-01-01T00:00:00,0,None,0,0,0,0,http://api.ratings.food.gov.uk/establishments/...
4,66ac5544b2f82719332244f9,694609,0,PI/000116619,Volunteer,Pub/bar/nightclub,7843,130 - 132 Plumstead High Street,,Plumstead,...,0,None,0001-01-01T00:00:00,0,None,0,0,0,0,http://api.ratings.food.gov.uk/establishments/...


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [146]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
match_query = {
    '$match': {
        'scores.Hygiene': 0
    }
}

# 2. Groups the matches by Local Authority
group_query = {
    '$group': {'_id':{'LocalAuthorityName':'$LocalAuthorityName'},
               'count':{'$sum': 1}
               }
}

# 3. Sorts the matches from highest to lowest
sort_values = {
    '$sort': {'count': -1}
}

# Put the pipeline together
pipeline = [match_query, group_query, sort_values]

# Run the pipeline
restaurant_by_location_results = list(establishments.aggregate(pipeline))

#print
print(f'There are {len(restaurant_by_location_results)} rows in results.')

There are 55 rows in results.


In [147]:
restaurant_by_location_results[0:10]

[{'_id': {'LocalAuthorityName': 'Thanet'}, 'count': 1130},
 {'_id': {'LocalAuthorityName': 'Greenwich'}, 'count': 882},
 {'_id': {'LocalAuthorityName': 'Maidstone'}, 'count': 713},
 {'_id': {'LocalAuthorityName': 'Newham'}, 'count': 711},
 {'_id': {'LocalAuthorityName': 'Swale'}, 'count': 686},
 {'_id': {'LocalAuthorityName': 'Chelmsford'}, 'count': 680},
 {'_id': {'LocalAuthorityName': 'Medway'}, 'count': 672},
 {'_id': {'LocalAuthorityName': 'Bexley'}, 'count': 607},
 {'_id': {'LocalAuthorityName': 'Southend-On-Sea'}, 'count': 586},
 {'_id': {'LocalAuthorityName': 'Tendring'}, 'count': 542}]

In [149]:
# Flatten the data
flatten_data = [
    {
        'LocalAuthorityName' : item['_id']['LocalAuthorityName'],
        'count': item['count']
    } for item in restaurant_by_location_results
]

# Convert the result to a Pandas DataFrame
restaurant_by_location_df = pd.DataFrame(flatten_data)
restaurant_by_location_df

,LocalAuthorityName,count
0,Thanet,1130
1,Greenwich,882
2,Maidstone,713
3,Newham,711
4,Swale,686
5,Chelmsford,680
6,Medway,672
7,Bexley,607
8,Southend-On-Sea,586
9,Tendring,542


In [150]:
# Display the number of rows in the DataFrame
num_restaurant_by_location_df = restaurant_by_location_df.shape[0]
print(f'There are {num_restaurant_by_location_df} rows in results.')

There are 55 rows in results.


In [151]:
# Display the first 10 rows of the DataFrame
restaurant_by_location_df.head(10)

,LocalAuthorityName,count
0,Thanet,1130
1,Greenwich,882
2,Maidstone,713
3,Newham,711
4,Swale,686
5,Chelmsford,680
6,Medway,672
7,Bexley,607
8,Southend-On-Sea,586
9,Tendring,542
